# Imports

In [ ]:
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.graph_objs as go

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

from torchvision.models import googlenet, GoogLeNet_Weights

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) 

# Load dataset

In [ ]:
resize = transforms.Resize(size=(128,128))
hFlip = transforms.RandomHorizontalFlip(p=0.25)
vFlip = transforms.RandomVerticalFlip(p=0.25)
rotate = transforms.RandomRotation(degrees=15)

In [ ]:
transform_train = transforms.Compose([resize, hFlip, vFlip, rotate, transforms.ToTensor()])
transform_test = transforms.Compose([resize, transforms.ToTensor()])

In [ ]:
trainDataset = ImageFolder(root="./data/Dataset/Train", transform=transform_train)
testDataset = ImageFolder(root="./data/Dataset/Test", transform=transform_test)
validDataset = ImageFolder(root="./data/Dataset/Validation", transform=transform_test)

print(f'[INFO] training dataset contains {len(trainDataset)} samples')
print(f'[INFO] testing dataset contains {len(testDataset)} samples')
print(f'[INFO] validation dataset contains {len(validDataset)} samples')

In [ ]:
BATCH_SIZE = 256

In [ ]:
trainLoader = DataLoader(trainDataset,batch_size=BATCH_SIZE, shuffle= True)
testLoader = DataLoader(testDataset,batch_size=BATCH_SIZE, shuffle= True)
validLoader = DataLoader(validDataset,batch_size=BATCH_SIZE, shuffle= True)

In [ ]:
gender_dict = {0: 'Female',
               1: 'Male'}

# Model

## setup

In [ ]:
weights = GoogLeNet_Weights.IMAGENET1K_V1
model = googlenet(weights=weights).to(device=device)

## Classifier training

In [ ]:
for param in model.parameters():
    param.requires_grad=False

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=1, bias=True)
activation = nn.Sigmoid()

In [ ]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

In [ ]:
NUM_EPOCH = 10

In [ ]:
loss_values = []
for epoch in range(NUM_EPOCH):
    total_loss = 0
    model.train()
    for X, label in trainLoader:
        optimizer.zero_grad()

        X = X.to(device=device)
        label = label.to(device=device)
        label = label.reshape(-1,1).float()
        
        logits = activation(model(X)).float()
        
        loss = loss_fn(logits, label)
        
        loss.backward()
        optimizer.step()
    
        total_loss += loss.item()
        
    avg_loss = total_loss / len(trainLoader)
    loss_values.append(avg_loss)
    print(f'Epoch: {epoch+1}/{NUM_EPOCH}, loss values: {avg_loss}')
        
        

In [ ]:
best_model = copy.deepcopy(model.state_dict())

## Testing method

In [ ]:
def test_model(model, dataLoader):
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X, label in dataLoader:
            X = X.to(device=device)
            label = label.to(device=device)
            label = label.reshape(-1,1).float()
        
            logits = activation(model(X)).float()
            
            predicted = logits.round()

            total += label.size(0)
            correct += (predicted == label).sum().item()

    accuracy = 100 * correct / total
    return accuracy
            

## Finetuning

In [ ]:
for param in model.parameters():
    param.requires_grad=True

In [ ]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

In [ ]:
model.load_state_dict(best_model)

In [ ]:
best_accuracy = -1
for epoch in range(NUM_EPOCH):
    for i in [0,1]:
        if i == 0:
            total_loss = 0
            model.train()
            for X, label in trainLoader:
                optimizer.zero_grad()

                X = X.to(device=device)
                label = label.to(device=device)
                label = label.reshape(-1,1).float()
                
                logits = activation(model(X)).float()
                
                
                    
                loss = loss_fn(logits, label)
                
                loss.backward()
                optimizer.step()
            
                total_loss += loss.item()
                
                
                
            avg_loss = total_loss / len(trainLoader)
            loss_values.append(avg_loss)
            print(f'Epoch: {epoch+1}/{NUM_EPOCH}, loss values: {avg_loss}')
        else:
            accuracy = test_model(model, testLoader)
            if accuracy > best_accuracy:
                best_model = copy.deepcopy(model.state_dict())
            lr_scheduler.step(accuracy)
            
            print(f'--- Accuracy of the model on the test data: {accuracy:.2f}% ---')
            
model.load_state_dict(best_model)